In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
from cycler import cycler
from IPython.display import display
import datetime
import scipy.stats as stats
import warnings
from colorama import Fore, Back, Style
import gc

from sklearn.model_selection import StratifiedKFold
from sklearn.calibration import CalibrationDisplay
import lightgbm
from lightgbm import LGBMClassifier, log_evaluation

plt.rcParams['axes.facecolor'] = '#e8e3e3' 
plt.rcParams['axes.prop_cycle'] = cycler(color=['#0c7bdc'] +
                                         plt.rcParams['axes.prop_cycle'].by_key()['color'][3:])
plt.rcParams['text.color'] = 'b'

INFERENCE = True 

In [2]:
def amex_metric(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)

def lgb_amex_metric(y_true, y_pred):
    """The competition metric with lightgbm's calling convention"""
    return ('amex',
            amex_metric(y_true, y_pred),
            True)

In [3]:
%%time
features_avg = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_8', 'B_9', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14',\
                'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_28',\
                'B_29', 'B_30', 'B_32', 'B_33', 'B_37', 'B_38', 'B_39', 'B_40', 'B_41',  'D_39', 'D_41',\
                'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_50', 'D_51', 'D_53', 'D_54', 'D_55',\
                'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_65', 'D_66', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73',\
                'D_74', 'D_75', 'D_76', 'D_77', 'D_78', 'D_80', 'D_82', 'D_84', 'D_91', 'D_92', 'D_96',\
                'D_103', 'D_104', 'D_108', 'D_112', 'D_113', 'D_114', 'D_115', 'D_117', 'D_118', 'D_119', 'D_120',\
                'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_126', 'D_128', 'D_129', 'D_131', 'D_132', 'D_133',\
                'D_134', 'D_135', 'D_136', 'D_140', 'D_141', 'D_142', 'D_144', 'D_145', 'P_2', 'P_3', 'P_4', 'R_1',\
                'R_2', 'R_3', 'R_7', 'R_8', 'R_9', 'R_10', 'R_11', 'R_14', 'R_15', 'R_16', 'R_17', 'R_20', \
                'R_22',  'R_26', 'R_27', 'S_3', 'S_5', 'S_6', 'S_7', 'S_9', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16',\
                'S_18', 'S_22', 'S_23', 'S_25', 'S_26', 'B_42','D_86','D_94','R_21','R_24']
features_min = ['B_2', 'B_4', 'B_5', 'B_9', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_19', 'B_20', 'B_28',\
                'B_29', 'B_36',  'D_41', 'D_42', 'D_45', 'D_46', 'D_48', 'D_50', 'D_51',\
                'D_53', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_62', 'D_71', 'D_74', 'D_75', \
                'D_102', 'D_112', 'D_113', 'D_115', 'D_118', 'D_119', 'D_121', 'D_122', 'D_128', 'D_132',\
                'D_141', 'D_144', 'D_145', 'P_2', 'P_3', 'R_1', 'R_27', 'S_3', 'S_5', 'S_7', 'S_9', 'S_11',\
                'S_12', 'S_23', 'S_25','B_42','B_33', 'D_39', 'D_78','D_83','D_70','D_140']
features_max = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'B_10', 'B_12', 'B_13', 'B_14', 'B_15',\
                'B_16', 'B_17', 'B_18', 'B_19', 'B_21', 'B_23', 'B_24', 'B_25', 'B_29', 'B_30', 'B_37', 'B_38',\
                'B_40', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49',\
                'D_50', 'D_52', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_63', 'D_64', 'D_65', 'D_70', 'D_71',\
                'D_72', 'D_73', 'D_74', 'D_76', 'D_77', 'D_78', 'D_80', 'D_82', 'D_84', 'D_102', 'D_105', 'D_107',\
                'D_110', 'D_112', 'D_115', 'D_116', 'D_117', 'D_118', 'D_119', 'D_121', 'D_122', 'D_123', 'D_124',\
                'D_125', 'D_128', 'D_131', 'D_132', 'D_133', 'D_134', 'D_138',  'D_141',\
                'D_142', 'D_144', 'D_145', 'P_2', 'P_3', 'P_4', 'R_1', 'R_3', 'R_5', 'R_6', 'R_7','R_10', 'R_11', \
                'R_14',  'R_20', 'R_26', 'R_27', 'S_3', 'S_5', 'S_7', 'S_8', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', \
                'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27','B_42','B_39','D_91',  'R_8', 'D_136','D_140','D_111','D_126','D_135','B_33','R_17']
features_last = ['B_1', 'B_2', 'B_3', 'B_4', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', \
                 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_28',\
                 'B_29', 'B_30', 'B_33', 'B_36', 'B_37', 'B_38', 'B_39', 'B_40', 'D_39', 'D_41',\
                 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54',\
                 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_63', 'D_64', 'D_65', 'D_69', 'D_70', 'D_71',\
                 'D_72',  'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80',  'D_82', 'D_83', 'D_86', 'D_91',\
                 'D_96', 'D_105', 'D_106', 'D_112', 'D_114', 'D_119', 'D_120', 'D_121', 'D_122', 'D_124',  \
                 'D_127', 'D_130', 'D_131', 'D_132', 'D_133', 'D_134', 'D_138', 'D_140', 'D_141', 'D_142', \
                 'D_145', 'P_2', 'P_3', 'P_4', 'R_1', 'R_2', 'R_3',  'R_5', 'R_6', 'R_7', 'R_9', 'R_10',\
                 'R_11', 'R_12', 'R_20', 'R_26', 'R_27', 'S_3', 'S_5', 'S_7', \
                 'S_8', 'S_9', 'S_11', 'S_12', 'S_13', 'S_16', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26',\
                 'S_27','B_32','B_41', 'B_42', 'D_73','D_81','D_125','D_126', 'R_4', 'R_14', 'R_8','R_13','R_15','R_19','S_6']
for i in ['test', 'train'] if INFERENCE else ['train']:
    df = pd.read_parquet(f'../input/amex-data-integer-dtypes-parquet-format/{i}.parquet')
    cid = pd.Categorical(df.pop('customer_ID'), ordered=True)
    last = (cid != np.roll(cid, -1)) # mask for last statement of every customer
    if 'target' in df.columns:
        df.drop(columns=['target'], inplace=True)
    gc.collect()
    print('Read', i)
    df_avg = (df
              .groupby(cid)
              .mean()[features_avg]
              .rename(columns={f: f"{f}_avg" for f in features_avg})
             )
    gc.collect()
    print('Computed avg', i)
    df_min = (df
              .groupby(cid)
              .min()[features_min]
              .rename(columns={f: f"{f}_min" for f in features_min})
             )
    gc.collect()
    print('Computed min', i)
    df_max = (df
              .groupby(cid)
              .max()[features_max]
              .rename(columns={f: f"{f}_max" for f in features_max})
             )
    gc.collect()
    print('Computed max', i)

    df = (df.loc[last, features_last]
          .rename(columns={f: f"{f}_last" for f in features_last})
          .set_index(np.asarray(cid[last]))
         )
    gc.collect()
    print('Computed last', i)
        
    #last["c_PB_29"]=(last["P_2_last"]*(-1)-.0001)/(last["B_9_last"]*(1)+0.0001)
    print('Computed extras', i)
    
    df = pd.concat([df, df_min, df_max,df_avg], axis=1)
    if i == 'train': train = df
    else: test = df
    print(f"{i} shape: {df.shape}")
    del df, df_avg, df_min, df_max, cid, last
    gc.collect()




    
target = pd.read_csv('../input/amex-default-prediction/train_labels.csv').target.values
print(f"target shape: {target.shape}")

Read test
Computed avg test
Computed min test
Computed max test
Computed last test
Computed extras test
test shape: (924621, 469)
Read train
Computed avg train
Computed min train
Computed max train
Computed last train
Computed extras train
train shape: (458913, 469)
target shape: (458913,)
CPU times: user 6min 39s, sys: 1min 36s, total: 8min 15s
Wall time: 8min 13s


In [4]:
test["c_DP_239"]=(test["D_39_last"]+.0001)/(test["P_2_last"]*(-1)+0.0001)
test["a_DP_239"]=(test["D_39_avg"]+.0001)/(test["P_2_avg"]*(-1)+0.0001)
test["c_PB_29"]=(test["P_2_last"]*(-1)+.0001)/(test["B_9_last"]*(1)+0.0001)
test["c_PR_21"]=(test["P_2_last"]*(-1)+.0001)/(test["R_1_last"]+0.0001)

test["c_DP_348"]=(test["D_48_last"]+.0001)/(test["P_3_last"]+0.0001)
test["c_DP_355"]=(test["D_55_last"]+.0001)/(test["P_3_last"]+0.0001)

test["c_DP_39r"]=(test["D_39_last"]+.0001)/(test["P_4_last"]+0.0001)
test["c_PB_49"]=(test["B_9_last"]+.0001)/(test["P_4_last"]+0.0001)
test["c_PR_41"]=(test["R_1_last"]+.0001)/(test["P_4_last"]+0.0001)
test["c_BB93"]=(test["B_9_last"]+0.001)/(test["B_23_last"]+test["B_3_last"]+0.0001)
test["c_BB1825"]=(test["B_33_last"]*(-1))+(test["B_18_last"]*(-1)+test["S_25_last"]*(1)+0.0001)
test["c_BB1920"]=(test["B_19_last"]+test["B_20_last"]+test["B_4_last"]+0.0001)

test["c_R3R2"]=(test["R_3_last"]+0.001)/(test["R_2_last"]+test["R_4_last"]+0.0001)
test["c_DDR6227"]=(test["D_62_last"]+0.001)/(test["D_112_last"]+test["R_27_last"]+0.0001)
print('^ Fixing those DataFrame fragmentation errors ^')
 
setter = test.copy()
del test
test = setter.copy()
del setter
gc.collect() 
print('Errors mitigated successfully, moving on...')
train["c_DP_239"]=(train["D_39_last"]+.0001)/(train["P_2_last"]*(-1)+0.0001)
train["a_DP_239"]=(train["D_39_avg"]+.0001)/(train["P_2_avg"]*(-1)+0.0001)
train["c_PB_29"]=(train["P_2_last"]*(-1)+.0001)/(train["B_9_last"]*(1)+0.0001)
train["c_PR_21"]=(train["P_2_last"]*(-1)+.0001)/(train["R_1_last"]+0.0001)

train["c_DP_348"]=(train["D_48_last"]+.0001)/(train["P_3_last"]+0.0001)
train["c_DP_355"]=(train["D_55_last"]+.0001)/(train["P_3_last"]+0.0001)

train["c_DP_39r"]=(train["D_39_last"]+.0001)/(train["P_4_last"]+0.0001)
train["c_PB_49"]=(train["B_9_last"]+.0001)/(train["P_4_last"]+0.0001)
train["c_PR_41"]=(train["R_1_last"]+.0001)/(train["P_4_last"]+0.0001)
train["c_BB93"]=(train["B_9_last"]+0.001)/(train["B_23_last"]+train["B_3_last"]+0.0001)
train["c_BB1825"]=(train["B_33_last"]*(-1))+(train["B_18_last"]*(-1)+train["S_25_last"]*(1)+0.0001)
train["c_BB1920"]=(train["B_19_last"]+train["B_20_last"]+train["B_4_last"]+0.0001)

train["c_R3R2"]=(train["R_3_last"]+0.001)/(train["R_2_last"]+train["R_4_last"]+0.0001)
train["c_DDR6227"]=(train["D_62_last"]+0.001)/(train["D_112_last"]+train["R_27_last"]+0.0001)

gc.collect()

print('^ Fixing those DataFrame fragmentation errors ^')
setter = train.copy()
del train
train = setter.copy()
del setter 
gc.collect() 
print('Errors mitigated successfully, moving on...')

^ Fixing those DataFrame fragmentation errors ^


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  import sys
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragme

Errors mitigated successfully, moving on...
^ Fixing those DataFrame fragmentation errors ^


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, u

Errors mitigated successfully, moving on...


In [5]:
print(train.shape)
train.head()

(458913, 483)


,B_1_last,B_2_last,B_3_last,B_4_last,B_5_last,B_6_last,B_7_last,B_8_last,B_9_last,B_10_last,...,c_DP_348,c_DP_355,c_DP_39r,c_PB_49,c_PR_41,c_BB93,c_BB1825,c_BB1920,c_R3R2,c_DDR6227
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.009382,1.007647,0.007174,5,0.231717,0.149564,0.058425,0.000000,0.009535,0.326101,...,0.305764,0.297677,1.000000,96.348511,62.037540,0.221129,-1.027906,5.0001,10.0,0.223089
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.034684,1.004028,0.005068,1,0.027000,0.167634,0.028411,0.000000,0.012926,0.297130,...,0.025912,0.063419,60001.001516,130.263092,70.111542,0.700775,-1.022226,1.0001,10010.0,0.117040
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.004284,0.812649,0.007196,2,0.001557,0.183628,0.026981,0.000000,0.009392,0.296313,...,0.127925,0.157484,1.000000,94.920845,65.499542,0.377568,-1.027532,2.0001,10.0,0.219482
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.012564,1.006183,0.009937,0,0.118818,0.174331,0.011969,1.005561,0.020526,0.411625,...,NaN,0.031986,1.000000,206.261841,79.287788,1.425827,-1.025859,0.0001,20010.0,0.217254
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.007679,0.815746,0.005528,21,0.004855,0.048857,0.159818,1.005185,0.004027,0.125195,...,0.570045,0.445502,1.000000,41.272617,13.473127,0.033328,-0.559481,21.0001,10.0,0.211179


In [6]:
z = np.abs(stats.zscore(train))
threshold = 4
fixer = np.where(z > threshold)
fixer
print(len(fixer[0]))
print(len(fixer[1]))

975672
975672


In [7]:
fixer = pd.DataFrame(fixer)
fixer.head()

,0,1,2,3,4,5,6,7,8,9,...,975662,975663,975664,975665,975666,975667,975668,975669,975670,975671
0,0,0,0,8,8,8,8,8,8,9,...,458902,458906,458908,458908,458908,458909,458909,458909,458909,458909
1,73,320,397,151,185,329,338,356,358,67,...,455,68,72,398,465,88,128,282,356,421


In [8]:
fixer.iloc[1].value_counts()

135    23208
323    17926
107    14962
74     14075
72     13575
       ...  
355       34
327       31
370       15
239       12
416        1
Name: 1, Length: 239, dtype: int64

In [9]:
#have to use a model that will account for those 234 features (nearly half) = lgbm or keras
fixer = []
gc.collect()

46

In [ ]:
%%time
# Cross-validation of the classifier

ONLY_FIRST_FOLD = False

#to train model on all features use the next line
features = [f for f in train.columns if f != 'customer_ID' and f != 'target' ]

def my_booster(random_state=1, n_estimators=1550):
    return LGBMClassifier(n_estimators=n_estimators,
                          learning_rate=0.03, reg_lambda=50,
                          min_child_samples=2400,
                          num_leaves=94,
                          colsample_bytree=0.19,
                          max_bins=517, random_state=random_state)
      
print(f"{len(features)} features")
score_list = []
y_pred_list = []
kf = StratifiedKFold(n_splits=5)
for fold, (idx_tr, idx_va) in enumerate(kf.split(train, target)):
    X_tr, X_va, y_tr, y_va, model = None, None, None, None, None
    start_time = datetime.datetime.now()
    X_tr = train.iloc[idx_tr][features]
    X_va = train.iloc[idx_va][features]
    y_tr = target[idx_tr]
    y_va = target[idx_va]
    
    model = my_booster()
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=UserWarning)
        model.fit(X_tr, y_tr,
                  eval_set = [(X_va, y_va)], 
                  eval_metric=[lgb_amex_metric],
                  callbacks=[log_evaluation(100)])
    X_tr, y_tr = None, None
    y_va_pred = model.predict_proba(X_va, raw_score=True)
    score = amex_metric(y_va, y_va_pred)
    n_trees = model.best_iteration_
    if n_trees is None: n_trees = model.n_estimators
    print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} | {str(datetime.datetime.now() - start_time)[-12:-7]} |"
          f" {n_trees:5} trees |"
          f"                Score = {score:.5f}{Style.RESET_ALL}")
    score_list.append(score)
    
    if INFERENCE:
        y_pred_list.append(model.predict_proba(test[features], raw_score=True))
        
    if ONLY_FIRST_FOLD: break # we only want the first fold
    
print(f"{Fore.GREEN}{Style.BRIGHT}OOF Score:                       {np.mean(score_list):.5f}{Style.RESET_ALL}")

In [11]:
def sigmoid(log_odds):
    return 1 / (1 + np.exp(-log_odds))

plt.figure(figsize=(10, 4))
plt.hist(sigmoid(y_va_pred[y_va == 0]), bins=np.linspace(0, 1, 101),
         alpha=0.5, density=True, label='0')
plt.hist(sigmoid(y_va_pred[y_va == 1]), bins=np.linspace(0, 1, 101),
         alpha=0.5, density=True, label='1')
plt.xlabel('y_pred')
plt.ylabel('density')
plt.title('OOF Prediction histogram', color='black')
plt.legend()
plt.show()

In [ ]:
if INFERENCE:
    sub = pd.DataFrame({'customer_ID': test.index,
                        'prediction': np.mean(y_pred_list, axis=0)})
    sub.to_csv('submission4.csv', index=False)
    display(sub)